In [1]:
import pandas as pd
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
import matplotlib as plt
%pylab inline

from sklearn.model_selection import train_test_split

Populating the interactive namespace from numpy and matplotlib


C:\Users\Jiheng\Anaconda3\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['plt']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [3]:
# data = pd.read_csv('../ISR_WhyWeWearBlack.csv')
# print(type(data['time'][0]))
# data[:1]

<class 'str'>


,Unnamed: 0,Ignorant,Spreader,Recovered,time
0,0,13613,0,0,2018-01-05 00:00:00


In [4]:
# data['day'] = pd.to_datetime(data['time']).dt.dayofyear 
# data['hour'] = pd.to_datetime(data['time']).dt.hour
# data = data.sort_values(by=['day', 'hour'])
# data['day'] -= (data['day'].iloc[0] - 1)

In [5]:
# print(data[:2])
# print(data[-2:])

   Unnamed: 0  Ignorant  Spreader  Recovered                 time  day  hour
0           0     13613         0          0  2018-01-05 00:00:00    1     0
1           1     13613         0          0  2018-01-05 01:00:00    1     1
      Unnamed: 0  Ignorant  Spreader  Recovered                 time  day  \
1870        1870         0         0      13613  2018-03-23 22:00:00   78   
1871        1871         0         0      13613  2018-03-23 23:00:00   78   

      hour  
1870    22  
1871    23  


In [5]:
# data = data[:168]

In [2]:
tag = 'WhiteandGold'
data = pd.read_csv('../ISR_%s_Trimmed.csv' %tag).reset_index().rename(columns={"index": "Hour"})
data.head()

,Hour,Ignorant,Spreader,Recovered,time
0,0,67637,5,226,2/27/2015 0:00
1,1,67637,5,226,2/27/2015 1:00
2,2,67637,5,226,2/27/2015 2:00
3,3,67637,5,226,2/27/2015 3:00
4,4,67637,5,226,2/27/2015 4:00


In [4]:
# normalize the number of unique users
data['Ignorant'] /= 67868
data['Spreader'] /= 67868
data['Recovered'] /= 67868

In [5]:
data['i_dot'] = data['Ignorant'].diff()
data['s_dot'] = data['Spreader'].diff()
data['r_dot'] = data['Recovered'].diff()
data['IS'] = data['Ignorant'] * data['Spreader']
data['-SS'] = - data['Spreader'] * data['Spreader']

data = data.iloc[1:] #eliminate the first row
data[:2]

,Hour,Ignorant,Spreader,Recovered,time,i_dot,s_dot,r_dot,IS,-SS
1,1,0.000015,1.085525e-09,4.906574e-08,2/27/2015 1:00,0.0,0.0,0.0,1.594021e-14,-1.178365e-18
2,2,0.000015,1.085525e-09,4.906574e-08,2/27/2015 2:00,0.0,0.0,0.0,1.594021e-14,-1.178365e-18


In [6]:
# two linear models - first model, beta
X =  - data[['IS']]
y = data['i_dot']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=4)

In [7]:
lm = linear_model.LinearRegression()
model = lm.fit(X,y)
print(np.round(lm.score(X,y), 3)) # returns the coefficient of determination R^2 of the prediction (r-squared)
print(np.round(lm.coef_, 3)) # returns the estimated parameters beta1, beta2, delta

0.93
[76555.912]


In [8]:
lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)
print(np.round(lm.score(X_train,y_train), 3)) # returns the coefficient of determination R^2 of the prediction (r-squared)
print(np.round(lm.coef_, 3)) # returns the estimated parameters beta1, beta2, delta

0.911
[85884.873]


In [9]:
y_pred = model.predict(X)
lin_mse = mean_squared_error(y_pred, y)
lin_rmse = np.sqrt(lin_mse)
print('Liner Regression RMSE: %.4f' % lin_rmse)

Liner Regression RMSE: 0.0000


In [10]:
y_pred = model.predict(X_test)
lin_mse = mean_squared_error(y_pred, y_test)
lin_rmse = np.sqrt(lin_mse)
print('Liner Regression RMSE: %.4f' % lin_rmse)

Liner Regression RMSE: 0.0000


---

In [16]:
# two linear models - first model, beta
X_train_2 = X[:49]
y_train_2 = y[:49]
X_test_2 = X[49:]
y_test_2 = y[49:]

In [17]:
lm = linear_model.LinearRegression()
model = lm.fit(X_train_2,y_train_2)
print(np.round(lm.score(X_train_2,y_train_2), 3)) # returns the coefficient of determination R^2 of the prediction (r-squared)
print(np.round(lm.coef_, 3)) # returns the estimated parameters beta1, beta2, delta

0.918
[74202.11]


In [18]:
y_pred_2 = model.predict(X_test_2)
lin_mse_2 = mean_squared_error(y_pred_2, y_test_2)
lin_rmse = np.sqrt(lin_mse_2)
print('Liner Regression RMSE: %.4f' % lin_rmse)

Liner Regression RMSE: 0.0000


In [33]:
beta = lm.coef_
data['beta_IS'] = data['IS'] * beta
data['s_dot-beta_IS'] = data['s_dot']- data['beta_IS']

In [34]:
# two linear models - first model, gamma
X = data[['-SS']]
y = data['s_dot-beta_IS']

lm = linear_model.LinearRegression()
model = lm.fit(X,y)
print(np.round(lm.score(X,y), 3)) # returns the coefficient of determination R^2 of the prediction (r-squared)
print(np.round(lm.coef_, 3)) # returns the estimated parameters beta1, beta2, delta

0.495
[2.439]


In [35]:
y_pred = model.predict(X)
lin_mse = mean_squared_error(y_pred, y)
lin_rmse = np.sqrt(lin_mse)
print('Liner Regression RMSE: %.8f' % lin_rmse)

Liner Regression RMSE: 0.01134553


---

In [50]:
beta = lm.coef_
data_2['beta_IS'] = data_2['IS'] * beta
data_2['s_dot-beta_IS'] = data_2['s_dot']- data_2['beta_IS']
data_2['-S'] = - data_2['S_count']

C:\Users\ji326777\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\ji326777\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\ji326777\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

In [51]:
# two linear models - first model, gamma
X = data_2[['-SS', '-S']]
y = data_2['s_dot-beta_IS']

lm = linear_model.LinearRegression()
model = lm.fit(X,y)
print(np.round(lm.score(X,y), 3)) # returns the coefficient of determination R^2 of the prediction (r-squared)
print(np.round(lm.coef_, 3)) # returns the estimated parameters beta1, beta2, delta

0.667
[3.420566e+03 3.620000e-01]


In [53]:
y_pred = model.predict(X)
lin_mse = mean_squared_error(y_pred, y)
lin_rmse = np.sqrt(lin_mse)
print('Liner Regression RMSE: %.8f' % lin_rmse)

Liner Regression RMSE: 0.00000022
